In [1]:
import cv2, random
import numpy as np
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox, colorchooser
from PIL import Image, ImageTk

class ImageCollageApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Photo Album")
        self.images = []
        self.image_refs = []
        self.image_ids = []  # 이미지 ID 리스트 추가
        self.collage_size = (800, 500)

        self.collage_canvas = tk.Canvas(
            self.root,
            width=self.collage_size[0],
            height=self.collage_size[1],
            bg="white",
        )

        self.collage_canvas.pack(fill="both")
        
        self.bottom_frame = tk.Frame(self.root, background="white", relief="flat", height=800, width=10)
        self.bottom_frame.pack(side="bottom", anchor="s", fill="none", expand=True)

        button_frame = tk.Frame(self.bottom_frame, width=6)
        button_frame.pack(side="bottom", anchor="center", fill="none", expand=True)
        
        self.btn_add_image = tk.Button(
            button_frame,
            text="사진 불러오기",
            command=self.add_images,
            font=("Noto Sans KR", 12, "bold"),
        )
        self.btn_add_image.pack(side = tk.LEFT, ipadx=2, ipady=2, padx=30, pady=8)
        
        # 1. 묶음 메뉴 버튼 생성
        self.menu_button = tk.Menubutton(
            button_frame,
            text="필터",
            font=("Noto Sans KR", 12, "bold"),
        )
        self.menu_button.pack(side=tk.LEFT, ipadx=2, ipady=2, padx=30, pady=8)

        self.menu = tk.Menu(self.menu_button, tearoff=0)  # tearoff=0은 메뉴를 분리하지 않음을 의미
        self.menu_button.configure(menu=self.menu)  # 메뉴를 메뉴버튼에 연결

        # "필터" 메뉴 아이템 추가
        self.menu.add_command(label="밝게", command=self.filter_bright)
        self.menu.add_command(label="흑백", command=self.filter_gray)
        
        self.btn_create_collage = tk.Button(
            button_frame,
            text="스티커",
            command=self.add_sticker,
            font=("Noto Sans KR", 12, "bold"),
        )
        self.btn_create_collage.pack(side=tk.LEFT, ipadx=2, ipady=2, padx=30, pady=8)
        
        # 2. 묶음 메뉴 버튼 생성
        self.menu_button = tk.Menubutton(
            button_frame,
            text="그림판",
            font=("Noto Sans KR", 12, "bold"),
        )
        self.menu_button.pack(side=tk.LEFT, ipadx=2, ipady=2, padx=30, pady=8)

        self.menu = tk.Menu(self.menu_button, tearoff=0)  # tearoff=0은 메뉴를 분리하지 않음을 의미
        self.menu_button.configure(menu=self.menu)  # 메뉴를 메뉴버튼에 연결
        
        # "그림판" 메뉴 아이템 추가
        self.brush_menu = tk.Menu(self.menu, tearoff=0) # 메뉴 아이템을 self.menu에 추가해야 함
        self.menu.add_cascade(label="브러쉬", menu=self.brush_menu) # add_cascade로 메뉴 추가
        self.brush_menu.add_command(label="색상", command=self.getColor)
        self.brush_menu.add_separator()
        self.brush_menu.add_command(label="두께", command=self.getWidth)
        self.menu.add_command(label="지우기", command=self.clearCanvas)
        
        self.btn_create_collage = tk.Button(
            button_frame,
            text="콜라주 만들기",
            command=self.create_collage,
            font=("Noto Sans KR", 12, "bold"),
        )
        self.btn_create_collage.pack(side=tk.LEFT, ipadx=2, ipady=2, padx=30, pady=8)
        
        self.drag_data = {"x": 0, "y": 0, "item": None}
        self.image_positions = []
        self.image_params = []  # 이미지 인덱스를 가지고 있는 리스트
        
        self.collage_canvas.bind("<ButtonPress-1>", self.on_press)
        self.collage_canvas.bind("<B1-Motion>", self.on_drag)
        self.collage_canvas.bind("<Double-Button-1>", self.edit_image) # 더블 클릭 이벤트 바인딩
        self.collage_canvas.bind("<ButtonRelease-1>", self.on_release)
        self.collage_canvas.bind("<B3-Motion>", self.draw)
        
        # 변수 초기화
        self.dragging = False
        self.roi_start = (0, 0)
        self.roi_end = (0, 0)
        
        # 캔버스 위에 그림을 그리기 위한 변수 추가
        self.drawing = False
        self.brush_color = "black"
        self.brush_size = 1
        
        self.stickers = []  # 스티커 ID 및 이미지를 저장할 리스트
        
    def on_press(self, event):
        # 이미지를 드래그하고 있는 경우에만 실행되도록 수정
        item = self.collage_canvas.find_closest(event.x, event.y)
        if item and isinstance(item, tuple) and len(item) > 0:
            self.drag_data["item"] = item[0]  # 이미지 아이템의 ID
            self.drag_data["x"] = event.x
            self.drag_data["y"] = event.y
        else:
            # 그림 그리기 위한 동작으로 변경
            self.drawing = True
            self.drag_data["x"] = event.x
            self.drat_data["y"] = event.y
        
    def on_drag(self, event):
        delta_x = event.x - self.drag_data["x"]
        delta_y = event.y - self.drag_data["y"]
        self.collage_canvas.move(self.drag_data["item"], delta_x, delta_y)
        self.drag_data["x"] = event.x
        self.drag_data["y"] = event.y
        self.update_image_positions()  # 이미지 위치 업데이트
        
    def on_release(self, event):
        if self.drag_data["item"] is None:
            # 그림 그리기 동작을 종료하도록 변경
            self.drawing = False
        else:
            self.drag_data["item"] = None
            self.drag_data["x"] = 0
            self.drag_data["y"] = 0
            self.update_image_positions()
  
    def edit_image(self, event):
        selected_item = self.collage_canvas.find_closest(event.x, event.y)[0]
         
        # 선택된 이미지의 인덱스 찾기
        selected_index = -1
        for i, item in enumerate(self.collage_canvas.find_all()):
            if item == selected_item:
                selected_index = i
                break
        
        # 선택된 이미지 인덱스 유효성 확인
        if selected_index >= 0 and selected_index < len(self.images):
            selected_image = self.images[selected_index]

            # 이미지 경로를 임시 파일로 저장하여 opencv로 이미지 표시
            temp_image_path = f"temp_selected_{selected_index}.png"
            selected_image.save(temp_image_path)

            # OpenCV로 이미지 표시 및 마우스 이벤트 처리
            image = cv2.imread(temp_image_path)
            cv2.imshow(f"Selected Image {selected_index}", image)

            
            # 트랙바 이벤트 핸들러 함수 정의
            def on_trackbar(val):
                # 이미지 크기를 비율에 맞게 조절
                new_width = int(selected_image.width * val / 100)
                new_height = int(selected_image.height * val / 100)
                resized_image = selected_image.resize((new_width, new_height))

                # 이미지를 임시 파일로 저장하여 OpenCV로 표시
                temp_image_path = f"temp_selected_{selected_index}.png"
                resized_image.save(temp_image_path)
                image = cv2.imread(temp_image_path)
                cv2.imshow(f"Selected Image {selected_index}", image)
                
                 # 캔버스에 이미지 크기 조절 후 위치 변경
                self.images[selected_index] = resized_image
                self.image_refs[selected_index] = ImageTk.PhotoImage(resized_image)
                self.collage_canvas.itemconfig(self.image_ids[selected_index], image=self.image_refs[selected_index])
                self.update_image_positions()
                
            # 트랙바 생성 밑 설정
            cv2.namedWindow(f"Selected Image {selected_index}")
            cv2.createTrackbar("Image Size", f"Selected Image {selected_index}", 100, 300, on_trackbar)
            default_val = 100
            cv2.setTrackbarPos("Image Size", f"Selected Image {selected_index}", default_val)

            # 마우스 이벤트 콜백 함수 등록
            cv2.setMouseCallback(f"Selected Image {selected_index}", self.on_mouse_events,
                             param=self.image_params[selected_index])  # 이미지 인덱스 전달

            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
        else:
            messagebox.showwarning("Error", "Invalid image selection")
            
    def on_mouse_events(self, event, x, y, flags, param):
        selected_index = int(param.split()[2]) # 이미지 인덱스 추출
        
        if event == cv2.EVENT_LBUTTONDOWN:
            self.roi_start = (x, y)
            self.roi_end = (x, y)
            self.dragging = True

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.dragging:
                self.roi_end = (x, y)
                temp_image_path = f"temp_selected_{selected_index}.png"
                selected_image = self.images[selected_index]
                selected_image.save(temp_image_path)

                image = cv2.imread(temp_image_path)
                x1, y1 = self.roi_start
                x2, y2 = (x, y)
                roi = image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]

                # 이미지를 새로운 ROI로 대체하여 Canvas에 표시
                new_image = Image.fromarray(cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB))
                cv2.rectangle(image, self.roi_start, (x, y), (255, 0, 0), 2)  # 직사각형 그리기
                cv2.imshow(f"Selected Image {selected_index}", image)

                new_resized_image = self.resize_image(new_image)
                new_image_ref = ImageTk.PhotoImage(new_resized_image)
                self.image_refs[selected_index] = new_image_ref  # 이미지 참조 업데이트
                self.collage_canvas.itemconfig(self.image_ids[selected_index], image=new_image_ref)

        elif event == cv2.EVENT_LBUTTONUP:
            self.dragging = False
            self.roi_end = (x, y)

            # ROI 추출 및 저장
            if self.roi_start != self.roi_end:
                selected_index = int(param.split()[2])  # 이미지 인덱스 추출
                selected_image = self.images[selected_index]
                temp_image_path = f"temp_selected_{selected_index}.png"
                selected_image.save(temp_image_path)
    
                image = cv2.imread(temp_image_path)
                x1, y1 = self.roi_start
                x2, y2 = self.roi_end
                roi = image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]
                cv2.imwrite(f"roi_selected_{selected_index}.png", roi)
            
                # 직사각형 그리기
                cv2.rectangle(image, self.roi_start, (x, y), (255, 255, 255), 2) 
                
                cv2.imshow(f"Selected Image {selected_index}", image)
                cv2.setMouseCallback(f"Selected Image {selected_index}", self.on_mouse_events, param=self.image_params[selected_index])
                cv2.waitKey(0)
            
                # 이미지를 새로운 ROI로 대체하여 Canvas에 표시
                new_image = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
                new_resized_image = self.resize_image(new_image)
                new_image_ref = ImageTk.PhotoImage(new_resized_image)
                self.image_refs[selected_index] = new_image_ref  # 이미지 참조 업데이트
                self.collage_canvas.itemconfig(self.image_ids[selected_index], image=new_image_ref)

                print(f"ROI Extracted and Saved for Image {selected_index}")
    
    def update_image_positions(self):
        self.image_positions.clear

        for item in self.collage_canvas.find_all():
            x, y = self.collage_canvas.coords(item)
            self.image_positions.append((x, y))
            
    def add_images(self):
        num_images = simpledialog.askinteger(
            "Number of Images", "Enter the number of images:"
        )

        if num_images is not None:
            file_paths = filedialog.askopenfilenames(
                filetypes=[("Image files", "*.png")]
            )

            if file_paths:
                for i in range(min(num_images, len(file_paths))):
                    file_path = file_paths[i]
                    image = Image.open(file_path)
                    resized_image = self.resize_image(image)
                    self.images.append(resized_image)
                    self.image_refs.append(ImageTk.PhotoImage(resized_image))
                    self.image_params.append(f"Selected Image {len(self.images) - 1}")  # 이미지 인덱스 추가
                    
                    if len(self.images) == num_images:
                        break

                self.update_canvas()
                
    def resize_image(self, image):
        img_width, img_height = image.size
        aspect_ratio = img_width / img_height

        if aspect_ratio > 1:
            new_width = self.collage_size[0] // 2
            new_height = int(new_width / aspect_ratio)
        else:
            new_height = self.collage_size[1] // 2
            new_width = int(new_height * aspect_ratio)

        return image.resize((new_width, new_height))

    def update_canvas(self):
        self.collage_canvas.delete("all")
        rows = simpledialog.askinteger("Number of Rows", "Enter the number of rows:")
        cols = simpledialog.askinteger("Number of Columns", "Enter the number of columns:")

        collage_width = self.collage_size[0] * cols // 2
        collage_height = self.collage_size[1] * rows // 2
        self.collage_canvas.config(width=collage_width, height=collage_height)

        x_offset, y_offset = 0, 0        
        
        for i, image_ref in enumerate(self.image_refs):
            img_id = self.collage_canvas.create_image(
                x_offset, y_offset, anchor=tk.NW, image=image_ref
            )
            self.image_ids.append(img_id)  # 이미지 ID 저장
            
            self.image_positions.append((x_offset, y_offset))
            x_offset += self.collage_size[0] // 2

            if (i + 1) % cols == 0:
                x_offset = 0
                y_offset += self.collage_size[1] // 2

    def filter_bright(self):
        for i in range(len(self.images)):
            temp_image_path = f"temp_seleted_{i}.png"
            self.images[i].save(temp_image_path)
            
            # OpenCV로 이미지 밝게
            image = cv2.imread(temp_image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            dst1 = cv2.add(image_rgb, (30, 30, 30, 0))
            
            # 이미지를 다시 PIL Image로 변환하여 표시할 수 있도록 함
            pil_image = Image.fromarray(dst1)
            resized_pil_image = self.resize_image(pil_image)
            new_image_ref = ImageTk.PhotoImage(resized_pil_image)
            self.image_refs[i] = new_image_ref # 이미지 참조 업데이트
            self.collage_canvas.itemconfig(self.image_ids[i], image=new_image_ref)       
    
    def filter_gray(self):
        for i in range(len(self.images)):
            temp_image_path = f"temp_selected_{i}.png"
            self.images[i].save(temp_image_path)
            
            # OpenCV로 이미지 흑백으로 읽기
            image = cv2.imread(temp_image_path, cv2.IMREAD_GRAYSCALE)
            
            # 이미지를 다시 PIL Image로 변환하여 표시할 수 있도록 함
            pil_image = Image.fromarray(image)
            resized_pil_image = self.resize_image(pil_image)
            new_image_ref = ImageTk.PhotoImage(resized_pil_image)
            self.image_refs[i] = new_image_ref # 이미지 참조 업데이트
            self.collage_canvas.itemconfig(self.image_ids[i], image=new_image_ref)
            
    def add_sticker(self):
        file_paths = filedialog.askopenfilenames(filetypes=[("Image files", "*.png")])

        if file_paths:
            for file_path in file_paths:
                self.add_sticker_to_canvas(file_path)
                
    def add_sticker_to_canvas(self, file_path):
        sticker_image = Image.open(file_path)
        resized_sticker = self.resize_image(sticker_image)
        sticker_tk = ImageTk.PhotoImage(resized_sticker)

        def on_drag_sticker(event):
            delta_x = event.x - self.drag_data["x"]
            delta_y = event.y - self.drag_data["y"]
            self.collage_canvas.move(sticker_id, delta_x, delta_y)
            self.drag_data["x"] = event.x
            self.drag_data["y"] = event.y

        def on_release_sticker(event):
            self.collage_canvas.unbind("<B1-Motion>")  # 이벤트 바인딩 해제

        # 마우스 이벤트 바인딩
        self.collage_canvas.bind("<B1-Motion>", on_drag_sticker)
        self.collage_canvas.bind("<ButtonRelease-1>", on_release_sticker)

        # 랜덤 위치에 스티커 추가
        random_x = random.randint(0, self.collage_size[0] - resized_sticker.width)
        random_y = random.randint(0, self.collage_size[1] - resized_sticker.height)

        sticker_id = self.collage_canvas.create_image(
            random_x, random_y, anchor=tk.NW, image=sticker_tk
        )
        
        # 스티커 ID와 이미지를 리스트에 추가
        self.stickers.append({
            "id": sticker_id,
            "image": sticker_tk,
            "position": (random_x, random_y)
        })
        
        # 캔버스 이미지 가져오기
        canvas_image = self.get_canvas_image()

         # 스티커 이미지를 OpenCV 형식으로 변환하여 합성
        sticker_cv = cv2.cvtColor(np.array(resized_sticker), cv2.COLOR_RGB2BGR)
        sticker_width, sticker_height = resized_sticker.width(), resized_sticker.height()

        # 스티커를 캔버스 이미지에 합성
        for y in range(sticker_height):
            for x in range(sticker_width):
                if random_y + y < self.collage_size[1] and random_x + x < self.collage_size[0]:
                            canvas_image[random_y + y, random_x + x] = sticker_cv[y, x]

        # 합성된 이미지를 캔버스에 적용
        self.update_canvas_image(canvas_image)
        
    def get_canvas_image(self):
        canvas_width = self.collage_canvas.winfo_width()
        canvas_height = self.collage_canvas.winfo_height()
        canvas_image = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
        return canvas_image

    def draw(self, event):
        if self.drawing:
            x1, y1 = event.x-1, event.y-1
            x2, y2 = event.x+1, event.y+1
            self.collage_canvas.create_oval(x1, y1, x2, y2, width=self.brush_size, fill=None, outline=self.brush_color)
    
    def getColor(self):
        color = colorchooser.askcolor()
        if color:
            self.brush_color = color[1]
            self.drawing = True  # 색상 선택 시 drawing 변수를 True로 설정
        
    def getWidth(self):
        self.brush_size = simpledialog.askinteger("선 두께 설정", "선 두께(1~10)를 입력하세요.", minvalue=1, maxvalue=10)
    
    def clearCanvas(self):
        self.collage_canvas.delete("all")
    
    def create_collage(self):
        if len(self.images) == 0:
            messagebox.showwarning("Warning", "Please add images first!")
            return

        # 전체 콜라주 이미지를 위한 배경 생성
        background = Image.new("RGB", self.collage_size, "white")

        for idx, image in enumerate(self.images):
            x_offset, y_offset = self.image_positions[idx]
            x_offset, y_offset = int(x_offset), int(y_offset)

            paste_box = (
                x_offset,
                y_offset,
                x_offset + image.width,
                y_offset + image.height,
            )

            background.paste(image, paste_box)

        # 스티커 추가
        for sticker in self.stickers:
            sticker_image = sticker["image"]
            x_offset, y_offset = sticker["position"]

            # 스티커의 좌표와 너비, 높이 정보를 포함하는 튜플 생성
            sticker_box = (
                x_offset,
                y_offset,
                x_offset + sticker_image.width(),  # 너비
                y_offset + sticker_image.height(),  # 높이
            )

            # 스티커를 배경 이미지에 추가
            background.paste(sticker_image, sticker_box)

        # 최종 콜라주 이미지를 파일로 저장
        final_image_path = "final_collage_image.png"
        background.save(final_image_path)

        # 최종 이미지를 보여주기
        final_image = Image.open(final_image_path)
        final_image.show()

if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("800x550")
    app = ImageCollageApp(root)
    root.mainloop()

2023-12-10 15:19:08.942 python[12412:338745] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2023-12-10 15:19:10.102 python[12412:338745] +[CATransaction synchronize] called within transaction
2023-12-10 15:20:17.674 python[12412:338745] +[CATransaction synchronize] called within transaction
2023-12-10 15:20:19.288 python[12412:338745] +[CATransaction synchronize] called within transaction
2023-12-10 15:20:19.396 python[12412:338745] +[CATransaction synchronize] called within transaction
2023-12-10 15:20:19.455 python[12412:338745] +[CATransaction synchronize] called within transaction
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/gim-yeseul/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/by/p5z7_g4n2bg27d_w8jpn7fjr0000gn/T/ipykernel_12412/994665988.py", line 368, in add_sticker
    self.add_sticker_to_canvas(file